# COMPASS Engine: Interactive Demo

## Clinical Ontology-driven Multi-modal Predictive Agentic Support System

This notebook shows how to run COMPASS on a participant folder and inspect outputs.

What you'll do:
- Optionally launch the web dashboard
- Run the pipeline in-notebook (CLI mode)
- Inspect the final reports (standard + deep phenotyping)

Prerequisites:
- Data under `data/pseudo_data` (demo) or `data/__FEATURES__/COMPASS_data`
- If using OpenAI, set `OPENAI_API_KEY`


In [1]:
import sys
import json
import ipywidgets as widgets
from IPython.display import display, Markdown, JSON, clear_output
from pathlib import Path
import subprocess
import time


def find_repo_root(start: Path) -> Path:
    for path in [start] + list(start.parents):
        if (path / "main.py").exists() and (path / "agents").exists():
            return path
    return start


repo_root = find_repo_root(Path.cwd())
if str(repo_root.parent) not in sys.path:
    sys.path.insert(0, str(repo_root.parent))

print(f"[*] Repo Root: {repo_root}")

from multi_agent_system.main import run_compass_pipeline
from multi_agent_system.config.settings import get_settings, LLMBackend

PSEUDO_DATA_ROOT = repo_root / "data" / "pseudo_data"
if not PSEUDO_DATA_ROOT.exists():
    PSEUDO_DATA_ROOT = repo_root / "data" / "__FEATURES__" / "COMPASS_data"

print(f"[OK] COMPASS Engine Loaded. Data Source: {PSEUDO_DATA_ROOT}")


[*] Repo Root: /Users/stijnvanseveren/PythonProjects/IIS_BIOBIZKAIA/INFERENCE_PIPELINE/multi_agent_system
[OK] COMPASS Engine Loaded. Data Source: /Users/stijnvanseveren/PythonProjects/IIS_BIOBIZKAIA/INFERENCE_PIPELINE/multi_agent_system/data/pseudo_data


## Quick Launch: Web Dashboard

Launch the live UI (`main.py --ui`) for step-by-step monitoring in your browser.
This uses the same Python environment as the notebook.


In [2]:
def launch_dashboard(b):
    clear_output()
    print("Launching dashboard on port 5005...")
    print("Open: http://127.0.0.1:5005")
    try:
        main_script = repo_root / "main.py"
        subprocess.Popen([sys.executable, str(main_script), "--ui"])
    except Exception as e:
        print(f"Error launching: {e}")

btn = widgets.Button(description="Launch Web Dashboard", button_style='info', icon='play')
btn.on_click(launch_dashboard)
display(btn)


Button(button_style='info', description='Launch Web Dashboard', icon='play', style=ButtonStyle())

## Interactive Notebook Execution

Run the pipeline directly in the notebook (CLI mode).
This is useful for quick iteration and debugging without the web dashboard.


### 1. Configuration
Select a subject, target condition, control comparator, backend, and number of iterations.
If you choose the Local backend, ensure the model is available on this machine.


In [ ]:
# Find available subjects
subjects = [d.name for d in PSEUDO_DATA_ROOT.iterdir() if d.is_dir() and not d.name.startswith('.')]
subjects.sort()

# Interactive Widgets
subject_dropdown = widgets.Dropdown(
    options=subjects,
    description='Subject:',
)

target_input = widgets.Text(
    value='Major Depressive Disorder',
    description='Target:',
    placeholder='e.g. Anxiety'
)

control_input = widgets.Text(
    value='brain-implicated pathology, but NOT psychiatric',
    description='Control:',
    placeholder='e.g. brain-implicated pathology, but NOT psychiatric'
)

iterations_input = widgets.BoundedIntText(
    value=2,
    min=1,
    max=5,
    description='Iterations:',
)

# Backend Selection
backend_dropdown = widgets.Dropdown(
    options=[('OpenAI API (gpt-5-nano)', 'openai'), ('Local LLM (vLLM/Transformers)', 'local')],
    value='openai',
    description='Backend:',
)

model_input = widgets.Text(
    value='Qwen/Qwen2.5-0.5B-Instruct',
    description='Local Model:',
    disabled=True
)


def on_backend_change(change):
    if change['new'] == 'local':
        model_input.disabled = False
    else:
        model_input.disabled = True


backend_dropdown.observe(on_backend_change, names='value')

display(subject_dropdown, target_input, control_input, iterations_input, backend_dropdown, model_input)


### 2. Run Analysis
This executes the pipeline and prints verbose output so you can follow agent reasoning.


In [ ]:
PARTICIPANT_DIR = PSEUDO_DATA_ROOT / subject_dropdown.value
TARGET = target_input.value
CONTROL = control_input.value
MAX_ITERS = iterations_input.value

print(f"Selected Participant: {PARTICIPANT_DIR}")

# Apply Backend Settings
settings = get_settings()
if backend_dropdown.value == 'local':
    settings.models.backend = LLMBackend.LOCAL
    settings.models.local_model_name = model_input.value
    print(f"Configured for LOCAL inference using {model_input.value}")
else:
    settings.models.backend = LLMBackend.OPENAI
    for attr in [
        'orchestrator_model', 'critic_model', 'predictor_model',
        'integrator_model', 'communicator_model', 'tool_model'
    ]:
        setattr(settings.models, attr, 'gpt-5-nano')
    print("Configured for OPENAI inference (gpt-5-nano for all agents)")

print(f"Starting analysis for: {TARGET}")
print(f"Control comparator: {CONTROL}")

# Run Pipeline
try:
    result = run_compass_pipeline(
        participant_dir=PARTICIPANT_DIR,
        target_condition=TARGET,
        control_condition=CONTROL,
        max_iterations=MAX_ITERS,
        verbose=True,
        interactive_ui=False  # Use CLI output mode for notebook compatibility
    )
except Exception as e:
    print(f"Execution Error: {e}")
    result = None


### 3. Inspector
Review the final standard report, the deep phenotyping report, and the structured execution log generated by the run.


In [ ]:
if result:
    log_path = Path(result['output_dir']) / f"execution_log_{result['participant_id']}.json"
    report_path = Path(result['output_dir']) / f"report_{result['participant_id']}.md"
    deep_path = Path(result['output_dir']) / "deep_phenotype.md"

    print(f"RESULT: {result['prediction']} ({result['probability']:.1%})")

    if report_path.exists():
        display(Markdown("### Final Clinical Report"))
        with open(report_path, 'r') as f:
            display(Markdown(f.read()))

    if deep_path.exists():
        display(Markdown("### Deep Phenotyping Report"))
        with open(deep_path, 'r') as f:
            display(Markdown(f.read()))

    if log_path.exists():
        display(Markdown("### Execution Log"))
        with open(log_path, 'r') as f:
            display(JSON(json.load(f), expanded=False))
else:
    print("No successful result to inspect.")
